In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
from IPython.display import HTML

In [2]:
import sys
import os

p = !pwd
p = os.path.dirname(p[0])
if p not in sys.path:
    sys.path.append(p)

In [3]:
from experanto.experiment import Experiment
from experanto.interpolators import Interpolator
from experanto.interpolators import ScreenInterpolator

root_folder = '../data/example_experiment'

e = Experiment(root_folder)

In [ ]:
# Example for images at the beginning
times = np.arange(300., 320., 0.1)
time_steps = len(times)
video, valid = e.interpolate(times, device="screen")

n_frames, height, width = video.shape

fig, ax = plt.subplots()
img = ax.imshow(video[0], cmap='gray', vmin=0, vmax=255)

def update(frame):
    img.set_array(video[frame])
    ax.set_title(f'Time step: {frame}')
    return [img]

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=50, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())

In [ ]:
# Example for video at the end
times = np.arange(4483., 4500., 0.1)
time_steps = len(times)
video, valid = e.interpolate(times, device="screen")

# can also do it using the dataset object using this
#video = dataset.__getitem__(4000)['screen']
#video = np.squeeze(video, axis=1)

n_frames, height, width = video.shape

fig, ax = plt.subplots()
img = ax.imshow(video[0], cmap='gray', vmin=0, vmax=255)

def update(frame):
    img.set_array(video[frame])
    ax.set_title(f'Time step: {frame}')
    return [img]

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=50, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())

In [17]:
from experanto.datasets import SimpleChunkedDataset, ChunkDataset
import torchvision
from torch.utils.data import DataLoader
from collections import OrderedDict


sampling_rate = 30  # Timestamps generated by this do not match the real ones, why do this? Is this for output data?
chunk_size = 20
dataset = SimpleChunkedDataset(root_folder, sampling_rate, chunk_size)

In [18]:
# This dataset object and the __getitem__ method can be used to sample data using the interpolators for all modalities at once
data = dataset.__getitem__(1000)
print(data.keys())
data['screen'].shape

dict_keys(['eye_tracker', 'screen', 'treadmill', 'responses', 'timestamps'])


(20, 1, 144, 256)

In [47]:
train_dataset = ChunkDataset(root_folder=root_folder, global_sampling_rate=sampling_rate,
            global_chunk_size=chunk_size,
            modality_config = 
            {'screen': {
                'sampling_rate': None,
                'chunk_size': None,
                'valid_condition': {
                    'tier': 'train',
                    'stim_type': 'stimulus.Frame', #include both images and videos
                    'stim_type': 'stimulus.Clip'
                },
                'offset': 0,
                'sample_stride': 4,
                # necessary for the allen dataset since there are blanks after every stimuli because else no valid times are found
                'include_blanks': True, 
                'transforms': {
                    'ToTensor': {
                        '_target_': 'torchvision.transforms.ToTensor'
                    },
                    'Normalize': {
                        '_target_': 'torchvision.transforms.Normalize',
                        'mean': 80.0,
                        'std': 60.0
                    },
                    'Resize': {
                        '_target_': 'torchvision.transforms.Resize',
                        'size': [144, 256]
                    },
                    'CenterCrop': {
                        '_target_': 'torchvision.transforms.CenterCrop',
                        'size': 144
                    }
                },
                'interpolation': {}
            },
            'responses': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0.1,
                'transforms': {
                    'standardize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            },
            'eye_tracker': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0,
                'transforms': {
                    'normalize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            },
            'treadmill': {
                'sampling_rate': None,
                'chunk_size': None,
                'offset': 0,
                'transforms': {
                    'normalize': True
                },
                'interpolation': {
                    'interpolation_mode': 'nearest_neighbor'
                }
            }
        })

In [51]:
data = train_dataset.__getitem__(10)
data['screen'].shape

torch.Size([20, 1, 144, 144])

In [53]:
batch_size = 1
data_loaders = OrderedDict()
m = 'example_experiment' 

data_loaders['train'] = OrderedDict()
data_loaders['train'][m] =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
it = next(iter(data_loaders['train'][m]))

In [63]:
it['treadmill'].shape

torch.Size([1, 20, 1])

In [59]:
it.keys()

dict_keys(['eye_tracker', 'screen', 'treadmill', 'responses', 'timestamps'])